In [ ]:
# scf_in = f'{mol_path}/dis_{1}/scf.in'
    
# with open(scf_in, 'r') as file:
#     file_content = file.readlines()
#     file.close()

# def parse_atom_coords(content):
#     atomic_positions = []
#     start = False
#     for line in content:
#         if 'ATOMIC_POSITIONS' in line:
#             start = True
#         elif start and line.strip():  # Continue reading until a blank line is encountered
#             if 'K_POINTS' in line:
#                 break
#             atomic_positions.append([float(coord) for coord in line.split()[1:]])

#     return np.array(atomic_positions)

# atomic_positions = parse_atom_coords(file_content)

In [ ]:
# projs = squared_projection(atomic_positions[0], atomic_positions[1])
# print(projs)

# def squared_projection(coords, i, squared_dist):
    
#     projs = []
#     squared_dist = np.sum((p1-p2)**2, axis=0)
#     for i in range(3):
#         proj = (p1[i] - p2[i])**2
#         s_proj = proj/squared_dist
#         projs.append(s_proj)
#     return projs

In [ ]:
def linal_sys_solve():
    coords = ['x', 'y', 'z']
    # fin_params = []
    all_ans = []
    all_coefs = []
    for j in tqdm(range(df_equat.shape[0])):
        for i in coords:
            all_ans.append(df_equat[f'polar_{i}'][j])
            coefs = [(1 - df_equat[f'g1_{i}'][j]), (1 - df_equat[f'g1_{i}'][j]) * df_equat.dr1[j],\
                       (1 - df_equat[f'g1_{i}'][j]) * df_equat.dr1_2[j], (1 - df_equat[f'g2_{i}'][j]),\
                       (1 - df_equat[f'g2_{i}'][j]) * df_equat.dr2[j], (1 - df_equat[f'g2_{i}'][j]) * df_equat.dr2_2[j],\
                  df_equat[f'g1_{i}'][j], df_equat[f'g1_{i}'][j]*df_equat.dr1[j], df_equat[f'g1_{i}'][j]*df_equat.dr1_2[j],\
                  df_equat[f'g2_{i}'][j], df_equat[f'g2_{i}'][j]*df_equat.dr2[j], df_equat[f'g2_{i}'][j]*df_equat.dr2_2[j]]
            all_coefs.append(coefs)
    np_coefs = np.array(all_coefs)

    x = np.linalg.lstsq(np_coefs, all_ans, rcond=None)[0]
    # pred = []
    # epss = []
    # for i in range(df_equat.shape[0]):
    #     pred.append((np.dot(np_coefs[i], x)/1000) + 1)
    #     epss.append((all_ans[i]/1000) + 1)
    #     # print(np.dot(np_coefs[i], x), all_ans[i])

    # print("Predicted  DFT")
    # for i in range(50):
    #     print(pred, '\t', epss)
    return x, np_coefs, all_ans
    # param_list = x[:(len(x)//2)]
    # fin_params.append(param_list)
        
    # return fin_params
    params, np_coefs, all_ans = linal_sys_solve()


In [ ]:
# Main

xx = 'epsxx.dat'
yy = 'epsyy.dat'
zz = 'epszz.dat'

def diag_eps_reader(type, inn_path):
    df = pd.read_table(f'{inn_path}/{type}', delimiter="\s+")
    eps = df.iloc[0,1]
    return eps


# Works only for system with two bonds
std_file_path = f'{mol_path}/SO2_0K.txt'
if os.path.exists(std_file_path):
    stan_coord = parse_atom_coords(file_path=std_file_path)
    r1 = np.sum((stan_coord[0]-stan_coord[1])**2, axis=0)
    r2 = np.sum((stan_coord[0]-stan_coord[2])**2, axis=0)
else:
    raise("Error: Standard file does not exists")

def collect_data():

    # eps_data = []
    polar_data = []
    equat_data = []
    
    for i in tqdm(range(l)):
        inn_path = f'{mol_path}/dis_{i}'
        dims = [xx, yy, zz, 'scf.in']
        if os.path.exists(inn_path) and all(os.path.exists(f'{inn_path}/{dim}') for dim in dims):
    
            # We collect here only epsilons
            
            eps_xx = diag_eps_reader(xx, inn_path)
            print(eps_xx)
            break
            # eps_yy = diag_eps_reader(yy)
            # eps_zz = diag_eps_reader(zz)
            # eps_data.append([eps_xx, eps_yy, eps_zz])
    
            # We collect here only polarization by axis
            polar_data = []
            for dim in dims[:3]:
                polar = diag_eps_reader(dim, inn_path) - 1
                polar_data.append(polar)
            # polar_xx = diag_eps_reader(xx, inn_path) - 1
            # polar_yy = diag_eps_reader(yy, inn_path) - 1 
            # polar_zz = diag_eps_reader(zz, inn_path) - 1
    
            atom_coords = parse_atom_coords(file_path=f'{inn_path}/scf.in')
            
            # We define or squared relation as "g"
            g1 = squared_projection(atom_coords[0], atom_coords[1])
            g2 = squared_projection(atom_coords[0], atom_coords[2])
    
            
            equat_data.append([polar_xx, polar_yy, polar_zz, *g1, *g2])
            # polar_data.append([polar_xx, polar_yy, polar_zz])
            # break
        else:
            pass
    
    # df_eps = pd.DataFrame(eps_data, columns=['eps_xx', 'eps_yy', 'eps_zz'])
    clmns = ['polar_xx', 'polar_yy', 'polar_zz', 'g1_x', 'g1_y', 'g1_z', 'g2_x', 'g2_y', 'g2_z']
    # clmns = ['polar_xx', 'polar_yy', 'polar_zz',\
    #          'g1_x', 'g1_y', 'g1_z', 'g2_x', 'g2_y', 'g2_z',\
    #         'drx', 'dry', 'drz']
    
    # df_polar = pd.DataFrame(polar_data, columns=['polar_xx', 'polar_yy', 'polar_zz]')
    df_equat = pd.DataFrame(equat_data, columns=clmns)
    return df_equat

df_equat = collect_data()

In [ ]:
def linal_sys_solve():
    coords = ['x', 'y', 'z']
    fin_params = []
    for k in range(df_equat.shape[0]//4):
        answers = []
        all_coefs = []
        for j in range(4*k, 4*k+4):
            # print(f'For {j} molecules')
            for i in coords:
                answers.append(df_equat[f'polar_{i}'][j])
                coefs = [(1 - df_equat[f'g1_{i}'][j]), (1 - df_equat[f'g1_{i}'][j]) * df_equat.dr1[j],\
                           (1 - df_equat[f'g1_{i}'][j]) * df_equat.dr1_2[j], (1 - df_equat[f'g2_{i}'][j]),\
                           (1 - df_equat[f'g2_{i}'][j]) * df_equat.dr2[j], (1 - df_equat[f'g2_{i}'][j]) * df_equat.dr2_2[j],\
                      df_equat[f'g1_{i}'][j], df_equat[f'g1_{i}'][j]*df_equat.dr1[j], df_equat[f'g1_{i}'][j]*df_equat.dr1_2[j],\
                      df_equat[f'g2_{i}'][j], df_equat[f'g2_{i}'][j]*df_equat.dr2[j], df_equat[f'g2_{i}'][j]*df_equat.dr2_2[j]]
                all_coefs.append(coefs)
        np_coefs = np.array(all_coefs)
        try:
        x = np.linalg.lstsq(np_coefs, answers, rcond=None)[0]
        # x = np.linalg.solve(np_coefs, answers)
        
        # np_coefs = []
        # answers = []
        # except:
        #     # pass
        #     print(k)
        #     answers = []
        #     all_coefs = []
        #     n = k-1
        #     print(df_equat.iloc[4*n:4*n+4, :])
        #     print(df_equat.iloc[4*k:4*k+4, :])
        #     u = k + 1
        #     print(df_equat.iloc[4*u:4*u+4, :])
            # for j in range(4*k, 4*k+4):
            #     for i in coords:
            #         answers.append(df_equat[f'polar_{i}'][j])
            #         coefs = [(1 - df_equat[f'g1_{i}'][j]), (1 - df_equat[f'g1_{i}'][j]) * df_equat.dr1[j],\
            #                    (1 - df_equat[f'g1_{i}'][j]) * df_equat.dr1_2[j], (1 - df_equat[f'g2_{i}'][j]),\
            #                    (1 - df_equat[f'g2_{i}'][j]) * df_equat.dr2[j], (1 - df_equat[f'g2_{i}'][j]) * df_equat.dr2_2[j],\
            #               df_equat[f'g1_{i}'][j], df_equat[f'g1_{i}'][j]*df_equat.dr1[j], df_equat[f'g1_{i}'][j]*df_equat.dr1_2[j],\
            #               df_equat[f'g2_{i}'][j], df_equat[f'g2_{i}'][j]*df_equat.dr2[j], df_equat[f'g2_{i}'][j]*df_equat.dr2_2[j]]
            #         all_coefs.append(coefs)
            # np_coefs = np.array(all_coefs)
            # print(np_coefs, len(np_coefs))
            # print()
            # g = np.unique(np_coefs)
            # print(g , len(g))
            # print(answers, len(answers))
            # break
        param_list = x[:(len(x)//2)]
        fin_params.append(param_list)
        
    return fin_params